<a href="https://colab.research.google.com/github/adisav17/ontology-driven-language-modeling/blob/master/knowlege_base_and_ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
class Entity:
    def __init__(self, name, is_common_noun = False, attributes=None):
        self.name = name
        self.is_common_noun = is_common_noun
        self.attributes = attributes if attributes else {}
        self.relationships = {}

    def add_relationship(self, relation_type, related_entity):
        self.relationships.setdefault(relation_type, set()).add(related_entity.name)


    def get_instances(self, relation_type='has_instance'):
        return list(self.relationships.get(relation_type, []))

    def add_instances(self, instances, ontology, relation_type='has_instance', is_common_noun=False):
        if not isinstance(instances, list):
            instances = [instances]  # If a single instance is provided, convert it to a list

        for instance_name in instances:
            instance_entity = ontology.entities.get(instance_name)
            if not instance_entity:
                instance_entity = Entity(name=instance_name, is_common_noun=is_common_noun)  # Create new entity if not found
                ontology.add_entity(instance_entity)  # Add the new entity to the ontology
                print(f"Entity {instance_name} created and added to the ontology.")

            self.add_relationship(relation_type, instance_entity)

    def __str__(self):
        return self.name


In [67]:
class Relationship:
    def __init__(self, relation_type, entity1, entity2):
        self.relation_type = relation_type
        self.entity1 = entity1
        self.entity2 = entity2
        entity1.add_relationship(relation_type, entity2)
        entity2.add_relationship(self.inverse_relation(relation_type), entity1)

    def inverse_relation(self, relation_type):
        # This method returns the inverse of the relation type.
        # For simplicity, we'll use a dictionary. This should be expanded upon.
        inverse_relations = {
            "subtype_of": "has_subtype",
            "instance_of": "has_instance"
        }
        return inverse_relations.get(relation_type, f"related_to_{relation_type}")


In [68]:
class Ontology:
    def __init__(self):
        self.entities = {}

    def add_entity(self, entity):
        if entity.is_common_noun:
            self.entities[entity.name] = entity
        else:
            print(f"Entity {entity.name} is not a common noun entity and can't be added to ontology.")

    def add_relationship(self, relation_type, entity1, entity2):
        #if entity1.is_common_noun and entity2.is_common_noun:
            Relationship(relation_type, entity1, entity2)
        #else:
        #    print(f"One or both entities are not common noun entities.")


    def get_hierarchy_list(self, entity, relation_type, hierarchy=None, visited=None):
        if hierarchy is None:
            hierarchy = []
        if visited is None:
            visited = set()

        if entity.name in visited:
            return hierarchy
        visited.add(entity.name)

        if relation_type in entity.relationships:
            for related_entity_name in entity.relationships[relation_type]:
                hierarchy.append(related_entity_name)
                related_entity = self.entities.get(related_entity_name)  # Fetch the actual entity object
                if related_entity:  # If the entity exists
                    self.get_hierarchy_list(related_entity, relation_type, hierarchy, visited)

        return hierarchy



    def get_hierarchy_tree(self, entity, relation_type, visited=None):

        hierarchy = []
        if visited is None:
            visited = set()

        if entity.name in visited:
            return hierarchy
        visited.add(entity.name)

        if relation_type in entity.relationships:
            for related_entity_name in entity.relationships[relation_type]:
                hierarchy.append(related_entity_name)
                related_entity = self.entities.get(related_entity_name)  # Fetch the actual entity object
                if related_entity:  # If the entity exists
                   # self.get_hierarchy(related_entity, relation_type, hierarchy, visited)
                   branch = self.get_hierarchy_tree(related_entity,relation_type,visited)
                   if branch is not None:
                       hierarchy.append(branch)


        return hierarchy



    def search_entities(self, criteria):
        # Placeholder for a method that will search entities based on provided criteria
        pass

    def update_relationship(self, entity, old_relation, new_relation):
        # Placeholder for a method to update relationships
        pass

    def delete_relationship(self, entity, relation_type):
        # Placeholder for a method to delete relationships
        pass

    def validate_entity(self, entity):
        # Placeholder for a method to validate entities before addition
        pass

    def visualize(self):
        # Placeholder for a method to visualize the ontology
        pass

    def export_ontology(self, format):
        # Placeholder for a method to export ontology to a specific format
        pass

    def import_ontology(self, data, format):
        # Placeholder for a method to import ontology from a specific format
        pass

    def display_hierarchy_list(self, entity, relation_type):
        hierarchy = self.get_hierarchy_list(entity, relation_type)
        print(" > ".join(reversed(hierarchy)) + f" > {entity.name}")

    def display_hierarchy_tree(self, entity, relation_type, indent=0):

        hierarchy_tree = self.get_hierarchy_tree(entity, relation_type)
        self._print_tree(hierarchy_tree, indent)

    def _print_tree(self, tree, indent):
        for item in tree:
            if isinstance(item, list):
                self._print_tree(item, indent + 2)
            else:
                print(' ' * indent + '- ' + item)








In [69]:
ontology = Ontology()

# Creating entities
profession = Entity(name="Profession", is_common_noun=True)
journalist = Entity(name="Journalist", is_common_noun=True)
athlete = Entity(name="Athlete", is_common_noun=True)
cricketer = Entity(name="Cricketer", is_common_noun=True)
# Adding entities to ontology
ontology.add_entity(profession)
ontology.add_entity(journalist)
ontology.add_entity(athlete)
ontology.add_entity(cricketer)
# Adding relationships
ontology.add_relationship("subtype_of", journalist, profession)
ontology.add_relationship("subtype_of", athlete, profession)
ontology.add_relationship("subtype_of", cricketer, athlete)


ontology.add_relationship("has_subtype",  profession, journalist)
ontology.add_relationship("has_subtype",  profession, athlete)
# Displaying hierarchical relationship
ontology.display_hierarchy_list(profession, "has_subtype")

Cricketer > Athlete > Journalist > Profession


In [70]:
ontology.display_hierarchy_tree(profession, "has_subtype")

- Journalist
- Athlete
  - Cricketer


In [71]:
ontology.get_hierarchy_list(profession, "has_subtype")

['Journalist', 'Athlete', 'Cricketer']

In [55]:
ontology.get_hierarchy_tree(profession, "has_subtype")

['Journalist', [], 'Athlete', ['Cricketer', []]]

In [72]:
list(profession.relationships["has_subtype"])

['Journalist', 'Athlete']

In [73]:

arnab = Entity(name="Arnab Goswami",is_common_noun=False,  attributes={})
rajdeep = Entity(name="Rajdeep Sardesai",is_common_noun=False,  attributes = {})
sachin = Entity(name = "Sachin Tendulkar", is_common_noun=False, attributes ={})
rahul = Entity(name = "Rahul Dravid", is_common_noun=False, attributes ={})



In [74]:

# Adding relationships
ontology.add_relationship("instance_of", arnab, journalist)
ontology.add_relationship("instance_of", rajdeep, journalist)
ontology.add_relationship("instance_of", sachin, cricketer)
ontology.add_relationship("instance_of", rahul, cricketer)

In [75]:
list(journalist.relationships['has_instance'])

['Arnab Goswami', 'Rajdeep Sardesai']

In [76]:
list(cricketer.relationships['has_instance'])

['Sachin Tendulkar', 'Rahul Dravid']

In [77]:
journalist.get_instances()

['Arnab Goswami', 'Rajdeep Sardesai']

In [78]:
journalist.add_instances(['Larry King'], ontology )



Entity Larry King is not a common noun entity and can't be added to ontology.
Entity Larry King created and added to the ontology.


In [79]:
journalist.get_instances()

['Arnab Goswami', 'Rajdeep Sardesai', 'Larry King']

In [ ]:
class DataImport:
    def __init__(self, folder_path, metadata_file):
        self.folder_path = folder_path
        self.metadata_file = metadata_file

    def import_entities(self, ontology):
        # logic to read entities from terminal folders and add to the ontology
        pass

    def import_relationships(self, ontology):
        # logic to read metadata file and establish relationships in the ontology
        pass

    def validate_structure(self):
        # logic to validate folder structure and metadata file format
        pass


In [82]:
import os

class DataExport:
    def __init__(self, folder_path, metadata_file):
        self.folder_path = folder_path
        self.metadata_file = metadata_file

    def export_entities(self, ontology):
        # logic to export entities from ontology to terminal folders
        pass

    def export_relationships(self, ontology):
        # logic to write relationships from ontology to metadata file
        pass

    def create_structure(self):
        # logic to create folder structure for exporting data
        pass

    def update_folder_structure(self, ontology):
        # Removes existing folder structure and recreates it based on the updated ontology
        # This could be optimized to only update the changed parts in a more complex implementation
        pass

        self._remove_existing_structure(self.folder_path)
        self.create_structure()

        self.export_entities(ontology)
        self.export_relationships(ontology)

    def _remove_existing_structure(self, path):
        # Logic to remove existing folder structure
        # Be very careful with this, ensure it doesn’t unintentionally delete important data
        pass

        for root, dirs, files in os.walk(path, topdown=False):
            for file in files:
                os.remove(os.path.join(root, file))
            for dir in dirs:
                os.rmdir(os.path.join(root, dir))


In [80]:
class Query:
    def get_entity(self, entity_name):
        # logic to retrieve entity details
        pass

    def get_relationship(self, entity1_name, entity2_name):
        # logic to retrieve relationship details
        pass

    def query(self, query_string):
        # logic to execute complex queries and return results
        pass


In [81]:
class Update:
    def add_entity(self, entity_name, attributes):
        # logic to add a new entity
        pass

    def update_entity(self, entity_name, updated_attributes):
        # logic to update an existing entity
        pass

    def delete_entity(self, entity_name):
        # logic to delete an entity and its relationships
        pass
